In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

In [4]:
url = 'https://raw.githubusercontent.com/thomaspingel/geodata/master/election/county_election_data_2000-2016.csv'
election_df = pd.read_csv(url,dtype={'FIPS':str})

url = 'https://raw.githubusercontent.com/thomaspingel/geodata/master/county_health_rankings/chr_2016.csv'
health_df = pd.read_csv(url,dtype={'FIPS':str})

df = health_df.merge(election_df,how='left',on='FIPS')

# df.columns = df.columns.str.replace(' ', '_')

df.head()

,FIPS,State,County,Premature death - # Deaths,Premature death - Years of Potential Life Lost Rate,Poor or fair health - % Fair/Poor,Poor physical health days - Physically Unhealthy Days,Poor mental health days - Mentally Unhealthy Days,Low birthweight - Unreliable,Low birthweight - # Low Birthweight Births,...,totalvotes_2012,gop_2012_prc,dem_2012_prc,gop_minus_dem_prc_2012,gop_2016_votes,dem_2016_votes,totalvotes_2016,gop_2016_prc,dem_2016_prc,gop_minus_dem_prc_2016
0,01001,Alabama,Autauga,809.0,9215.1,19.4,4.4,4.3,NaN,407.0,...,23932.0,72.62,26.59,46.03,18172.0,5936.0,24973.0,72.77,23.77,49.00
1,01003,Alabama,Baldwin,2506.0,7454.6,16.0,4.1,4.2,NaN,1300.0,...,85338.0,77.36,21.59,55.77,72883.0,18458.0,95215.0,76.55,19.39,57.16
2,01005,Alabama,Barbour,410.0,8499.9,25.7,4.9,4.7,NaN,272.0,...,11509.0,48.22,51.37,-3.15,5454.0,4871.0,10469.0,52.10,46.53,5.57
3,01007,Alabama,Bibb,414.0,11007.7,22.0,4.8,4.6,NaN,221.0,...,8420.0,72.83,26.15,46.68,6738.0,1874.0,8819.0,76.40,21.25,55.15
4,01009,Alabama,Blount,876.0,8655.9,20.7,4.6,4.5,NaN,364.0,...,24006.0,86.47,12.37,74.10,22859.0,2156.0,25588.0,89.33,8.43,80.90


In [5]:
print(df.columns.values)

['FIPS' 'State' 'County' 'Premature death - # Deaths'
 'Premature death - Years of Potential Life Lost Rate'
 'Poor or fair health - % Fair/Poor'
 'Poor physical health days - Physically Unhealthy Days'
 'Poor mental health days - Mentally Unhealthy Days'
 'Low birthweight - Unreliable'
 'Low birthweight - # Low Birthweight Births'
 'Low birthweight - # Live Births' 'Low birthweight - % LBW'
 'Adult smoking - % Smokers' 'Adult obesity - % Obese'
 'Food environment index - Food Environment Index'
 'Physical inactivity - % Physically Inactive'
 'Access to exercise opportunities - % With Access'
 'Excessive drinking - % Excessive Drinking'
 'Alcohol-impaired driving deaths - # Alcohol-Impaired Driving Deaths'
 'Alcohol-impaired driving deaths - # Driving Deaths'
 'Alcohol-impaired driving deaths - % Alcohol-Impaired'
 'Sexually transmitted infections - # Chlamydia Cases'
 'Sexually transmitted infections - Chlamydia Rate'
 'Teen births - Teen Births' 'Teen births - Teen Population'
 'Teen

# Run Model to Predict 2016 Results

In [17]:
_df = pd.DataFrame()

_df['y'] = df['gop_minus_dem_prc_2016']

_df['x0'] = df['gop_minus_dem_prc_2012']
_df['x1'] = df['totalvotes_2016']

_df = _df.dropna(how='any')

# Then run the regression
results = smf.ols(formula='y ~ x0 + x1',data=_df).fit()

# Print the report
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.903
Model:                            OLS   Adj. R-squared:                  0.903
Method:                 Least Squares   F-statistic:                 1.446e+04
Date:                Thu, 29 Oct 2020   Prob (F-statistic):               0.00
Time:                        13:53:24   Log-Likelihood:                -11453.
No. Observations:                3112   AIC:                         2.291e+04
Df Residuals:                    3109   BIC:                         2.293e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.9254      0.234     55.253      0.0

In [11]:
results.get_prediction

<bound method RegressionResults.get_prediction of <statsmodels.regression.linear_model.OLSResults object at 0x000001BC9954AFA0>>

In [12]:
result = pd.DataFrame()
result['model'] = results.fittedvalues

0       55.816137
1       63.040915
2        9.643170
3       56.892162
4       82.388206
          ...    
3136    49.740341
3137     1.368932
3138    68.003996
3139    66.137949
3140    80.478202
Length: 3112, dtype: float64